In [ ]:
# For translation
!pip install -U deep-translator

In [ ]:
# For sentiment analysis
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

In [ ]:
import tweepy
import pandas as pd
from datetime import datetime
import json
from dateutil import parser

Some Parameters to Be Initiated Before Running the Code

In [ ]:
# If you want to track users set this to 1, if you want to track by hashtags set to 0
TrackUsers = 0
# If you want to do sentiment analysis set this to 1
SentimentAnalysis = 1
# If you want to create a new collection entity, set this to 0 and add the collection description
CreateCollection = 1
CollectionDescription = 'This is a collection of tweets and sentiments about the traffic in Madrid'
# If you want the recovered tweets to belong to a collection, set this to 1
BelongsToCollection = 1

Crawling Tweets

In [ ]:
api_key = ""
api_key_secret= ""
access_token = ""
access_token_secret = ""

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
# Define the hashtags or keywords to search for
hashtags = ["#traffic", "#globalwarming"]

# Define the users to search for
users = ["@LeoDiCaprio"]

In [ ]:
# Define the number of objects you want returned
limit_hashtagtweets = 1
limit_usertimeline = 1
tweets = []

In [ ]:
# Historical tweet crawling based on keywords (or hashtags)
if not TrackKeywords:
  for hashtag in hashtags:
      tweets_hashtag = tweepy.Cursor(api.search_tweets,
                                    q=hashtag,
                                    tweet_mode='extended').items(limit_hashtagtweets)
      for tweet in tweets_hashtag:
          tweets.append(tweet)

In [ ]:
# Collect historical tweets posted by the users
if TrackUsers:
  for user in users:
      tweets_user = tweepy.Cursor(api.user_timeline,
                                  screen_name=user,
                                  # since_id=start_date,
                                  # until=end_date,
                                  tweet_mode='extended').items(limit_usertimeline)
      for tweet in tweets_user:
          tweets.append(tweet)

A Function to Hash the Input and Create IDs

In [ ]:
import hashlib

def HashSentence(sentence):
    # Convert the sentence to bytes since hashlib works with bytes
    sentence_bytes = sentence.encode('utf-8')

    # Create a SHA-256 hash object
    sha256_hash = hashlib.sha256()

    # Update the hash object with the sentence bytes
    sha256_hash.update(sentence_bytes)

    # Get the hexadecimal representation of the hash
    hashed_id = sha256_hash.hexdigest()

    return str(hashed_id)

def HashID(id2hash):
    # Convert the id to bytes since hashlib works with bytes
    id_bytes = str(id2hash).encode('utf-8')

    # Create a SHA-256 hash object
    sha256_hash = hashlib.sha256()

    # Update the hash object with the id bytes
    sha256_hash.update(id_bytes)

    # Get the hexadecimal representation of the hash
    hashed_id = sha256_hash.hexdigest()

    return str(hashed_id)

NGSI-LD Tweet, User, and Collection Entity Creation Functions

In [ ]:
def MapTweet2NGSILD (tweet, generated_user_id = 'NA', generated_post_id = 'NA', generated_collection_id = 'NA', generated_dataset_id = '0'):
    if generated_post_id == 'NA':
      generated_post_id = HashID(tweet['id'])
    if generated_user_id == 'NA':
      generated_user_id = HashID(tweet['user']['id'])
    tweet_entity = {
        "id": "urn:ngsi-ld:SMPost:" + generated_post_id,
        "type": "SMPost",
        "belongsToCollection": [
        ],

        "createdBy": {
            "type": "Relationship",
            "object": "urn:ngsi-ld:SMUser:" + generated_user_id
        },

        "hasAnalysis": [
        ],

        "platform": {
            "type": "Property",
            "value": "Twitter"
        },

        "postCreatedAt": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": parser.parse(tweet['created_at']).isoformat()
            }
        },

        "postId": {
            "type": "Property",
            "value": tweet['id_str']
        },

        "@context": [
            "https://raw.githubusercontent.com/smart-data-models/dataModel.SocialMedia/master/context.jsonld"
        ]
    }

    # Check if generated_collection_id is not 'NA' before adding to "belongsToCollection"
    if generated_collection_id != 'NA':
        tweet_entity["belongsToCollection"].append({
            "type": "Relationship",
            "object": "urn:ngsi-ld:SMCollection:" + generated_collection_id,
            "datasetId": "urn:ngsi-ld:Dataset:SMCollection:" + generated_dataset_id
        })

    return tweet_entity

In [ ]:
def CreateUserEntity(tweet, generated_user_id = 'NA', generated_post_id = 'NA', generated_dataset_id = '0'):
    user = tweet['user']
    if generated_user_id == 'NA':
      generated_user_id = HashID(user['id'])
    if generated_post_id == 'NA':
      generated_post_id = HashID(tweet['id'])

    user_entity = {
        "id": "urn:ngsi-ld:SMUser:" + generated_user_id,
        "type": "SMUser",

        "createdPosts": [
            {
                "type": "Relationship",
                "object": "urn:ngsi-ld:SMPost:" + generated_post_id,
                "datasetId": "urn:ngsi-ld:Dataset:SMPost:" + generated_post_id
            }
        ],

        "platform": {
            "type": "Property",
            "value": "Twitter"
        },

        "userId": {
            "type": "Property",
            "value": user['id_str']
        },

        "@context": [
            "https://raw.githubusercontent.com/smart-data-models/dataModel.SocialMedia/master/context.jsonld"
        ]
    }

    return user_entity

In [ ]:
def CreateCollectionEntity(generated_collection_id = 'NA', description = 'No Description Added'):

    if generated_collection_id == 'NA':
      generated_collection_id = HashSentence(description)

    collection = {
        "id": "urn:ngsi-ld:SMCollection:" + generated_collection_id,
        "type": "SMCollection",
        "description": {
            "type": "Property",
            "value": description
        },
        "groupedAt": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": datetime.now().isoformat()
            }
        },
        "hasAnalysis": [
        ],
        "hasPosts": [
        ],

        "@context": [
            "https://raw.githubusercontent.com/smart-data-models/dataModel.SocialMedia/master/context.jsonld"
        ]
    }

    return collection

NGSI-LD Sentiment Analysis Entity Creation

In [ ]:
# Creating a translator funcion to first translate tweets and then apply sentiment analysis

from deep_translator import GoogleTranslator

def GenerateTranslation (text_to_translate):
  translated_text = GoogleTranslator(source='auto', target='en').translate(text_to_translate)
  return translated_text

In [ ]:
# Creating a sentiment analysis pipeline

from transformers import pipeline
# specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
# specific_model(data)
sentiment_analyzer = pipeline('sentiment-analysis')

In [ ]:
from datetime import datetime

def CreateSentimentAnalysisEntity(tweet, translated_text, PostId = 'NA', generated_analysis_id = 'NA'):

    if PostId == 'NA':
      PostId = 'urn:ngsi-ld:SMPost:' + HashID(tweet['id'])
    if generated_analysis_id == 'NA':
      generated_analysis_id = HashID(tweet['id'])

    text_sentiment = sentiment_analyzer(translated_text)
    analysis = {
        "id": "urn:ngsi-ld:Analysis:Sentiment:" + generated_analysis_id,
        "type": "SMAnalysis",
        "analyzedAt": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": datetime.now().isoformat()
            }
        },
        "hasAnalysisType": {
            "type": "Property",
            "value": "Sentiment"
        },
        "hasAnalysisValue": {
            "type": "Property",
            "value": text_sentiment[0]['label']
        },
        "hasConfidence": {
            "type": "Property",
            "value": text_sentiment[0]['score']
        },
        "isAnalysisOf": {
            "type": "Relationship",
            "object": PostId
        },
        "@context": [
            "https://raw.githubusercontent.com/smart-data-models/dataModel.SocialMedia/master/context.jsonld"
        ]
    }

    return analysis

Crawl Tweets and Create the Entities

In [ ]:
if CreateCollection:
  ngsild_collection = CreateCollectionEntity(HashSentence(CollectionDescription), CollectionDescription)

# Map each tweet object to a NGSI-LD entity
for tweet in tweets:

  # SINCE WE HAVE ACCESS TO TWEET TEXT, HERE IS WHERE WE CAN FILTER THE TWEETS AND NOISES!

  ngsild_tweet = MapTweet2NGSILD(tweet)
  ngsild_user = CreateUserEntity(tweet)

  if SentimentAnalysis:
    translated_text = GenerateTranslation(tweet['full_text'])
    ngsild_sentiment = CreateSentimentAnalysisEntity(tweet, translated_text)

    # Save the NGSI-LD entities to a JSON file
    with open(f"{ngsild_sentiment['id']}.json", "w") as f:
      f.write(json.dumps(ngsild_sentiment, indent=4))

    new_relationship = {
        "type": "Relationship",
        "object": ngsild_sentiment['id'],
        "datasetId": "urn:ngsi-ld:Dataset:SMCollection:0"}
    ngsild_tweet['hasAnalysis'].append(new_relationship)

  if BelongsToCollection:
    new_relationship = {
        "type": "Relationship",
        "object": ngsild_tweet['id'],
        "datasetId":  "urn:ngsi-ld:Dataset:SMCollection:0"
    }
    ngsild_collection['hasPosts'].append(new_relationship)

    if SentimentAnalysis:
      new_relationship = {
        "type": "Relationship",
        "object": ngsild_sentiment['id'],
        "datasetId":  "urn:ngsi-ld:Dataset:SMCollection:0"
        }
      ngsild_collection['hasAnalysis'].append(new_relationship)

  # # Save the NGSI-LD entities to a JSON file
  with open(f"{ngsild_tweet['id']}.json", "w") as f:
    f.write(json.dumps(ngsild_tweet, indent=4))

  with open(f"{ngsild_user['id']}.json", "w") as f:
    f.write(json.dumps(ngsild_user, indent=4))

if CreateCollection:
  with open(f"{ngsild_collection['id']}.json", "w") as f:
    f.write(json.dumps(ngsild_collection, indent=4))